In [1]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.utils import *
from fgc_support_retri import config

In [3]:
data = json_load(config.FGC_TRAIN)

In [18]:
def get_fold_datas(data, fold):
    fold_len = int(len(data)/fold)
    fold_datas = []
    for i in range(fold):
        if i == fold-1:
            fold_data = data[fold_len*i:]
        else:
            fold_data = data[fold_len*i:fold_len * (i+1)]
        fold_datas.append(fold_data)
    return fold_datas

In [19]:
fold_datas = get_fold_datas(data, 10)

In [20]:
from fgc_support_retri.train import *

def train_entity_match_model(num_epochs, batch_size, model_file_name, train_documents):
    dataset_reader = CrossSentDataset
    
    tokenizer = BertTokenizer.from_pretrained(bert_model_name)
    pretrained_bert = BertModel.from_pretrained(bert_model_name)
    pretrained_bert.eval()
    
    model = EntityMatchModel.from_pretrained(bert_model_name)
    
    collate_fn = Sent_collate
    indexer = SentIdx(tokenizer, pretrained_bert)
    input_names = ['input_ids', 'token_type_ids', 'attention_mask',
                   'tf_type', 'idf_type', 'sf_score', 'atype_ent_match', 'label']
    trainer = SER_Trainer(model, collate_fn, indexer, dataset_reader, input_names)
    trainer.train(num_epochs, batch_size, model_file_name, train_documents)

In [ ]:
for i, train_documents in enumerate(fold_datas):
    train_entity_match_model(10, 12, "fold_{}_entity_match".format(i), train_documents)